In [1]:
from PIL import Image

import numpy as np

import cv2
import torch
import torchvision
import torch.nn as nn
import torchvision.datasets as datasets
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

import torch.optim as optim

# FCN

Avui farem feina amb xarxes que no tenen cap tipus de capa _fully connected_ per tant serà una xarxa _Fully Convolutional Network_ (FCN). Quan parlam d'una xarxa FCN, ens referim a xarxes tipus VGG. Ens anirà molt bé fer aquesta pràctica per poder passar a xarxes que fan segmentació ja que la meitat d'aquestes és una FCN.

Emprarem un dataset propi per fer aquesta pràctica. Això implica fer una mica més de feina per preparar les dades. En concret emprarem una versió del conjunt de dades : AIXI_SHAPE propi d'en Miquel Miró. [Enllaç](https://uibes-my.sharepoint.com/:u:/g/personal/gma040_id_uib_eu/EcsNAK5mkXRBqayDo1JYeooBWCf1lpRA-YJHT_kDF4J_nA?e=apkCql)

La feina d'avui és "lliure" (considerau-ho una mini-pràctica), el conjunt de dades que teniu a la vostra disposició permet fer com a mínim 4 feines:

1. **Regressió**: Contar quants d'objectes hi ha
2. **Regressió de classe**: Contar quants d'objectes de cada classe hi ha en una imatge.
3. **Detecció**: Mostrar on hi ha cada un dels objectes. Es podrien emprar xarxes ja fetes per aquesta tasca (tant les que teniu disponibles a pytorch com altres que trobeu)
4. **Segmentació**: Encara no en sabem, però ho resoldrem la setmana que vé.

Avui heu de fer una de les dues primeres. Tant podeu triar fer-ho amb les imatges amb textura, com amb les imatges binaries que serveixen com a _ground truth_ (gt).

Les imatges del gt són imatges binàries (0,1) de 3 canals on a cada canal hi ha un tipus d'objecte . Per poder contar el nombre d'objectes possiblement haureu de emprar les funcions `cv2.add` per unir tots els canals en una sola imatge i la funció `cv2.findContours` per contar el nombre d'objectes en una imatge. A més podeu demanar-me ajuda a mi o al vostre amic ChatGPT.


#### [Inciso] Si emprau Colab:

Aquest codi us serveix per connectar colab amb google drive:

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%ls
%cd #TODO al vostre sistema de fitxers


## Preparació de les Dades
Per preparar el conjunt de dades necessitarem fer algunes pases:

1. Crear una llista amb les imatges 
2. Crear una classe que ens permeti obtenir una tupla (imatge, etiqueta)
3. Emprar els objectes DataLoader com hem fet sempre, aquí no trobareu cap canvi

#### Crear una llista amb les imatges 


In [2]:
import os

path_train = "aixi_shape_256_texture/train/"

files = os.listdir(path_train)
img_files = list([f"{path_train}{p}" for p in files if p.endswith('.png')])
label_files = list([f"{path_train}gt/{p}" for p in files if p.endswith('.png')])

import os

path_test = "aixi_shape_256_texture/val/"

files = os.listdir(path_test)
test_img_files = list([f"{path_test}{p}" for p in files if p.endswith('.png')])
test_label_files = list([f"{path_test}gt/{p}" for p in files if p.endswith('.png')])

#### Crear una classe que ens permeti obtenir una tupla (imatge, etiqueta)

Aquesta classe hereta de la superclasse _Dataset_ i com a mínim ha de tenir els mètodes:

1. `__len__(self)`: retorna la longitud del dataset
2. `__getitem__(self, index)`: retorna l'element que es troba a la posició marcada pel valor d'index. Quan parlam d'un element parlam de la imatge i de la seva etiqueta.

El constructor i els atributs de la classe els he decidit jo:

- Llista amb els _paths_ a les imatges
- Llista amb els _paths_ a les imatges de gt que ens serviràn per calcular l'etiqueta de la imatge
- Un objecte transform

A la classe podeu afegir tants mètodes públics i privats com necessiteu

In [3]:

# Constructor del dataset.
class AIXI_Shape(Dataset):
    def __init__(self, images, labels, transform):
        super().__init__()
        self.paths = images
        self.labels = labels
        self.len = len(self.paths)
        self.transform = transform

    def __len__(self): 
        return self.len

    def __getitem__(self, index):
        image = cv2.imread(self.paths[index])
        image = cv2.resize(image, (224, 224), interpolation = cv2.INTER_AREA)
        
        # TODO: verificar? No fa res?
        gt_image = cv2.imread(self.labels[index])
        b,g,r = cv2.split(gt_image)
        gt_image = cv2.add(b,g,r)
        
        contours, _= cv2.findContours(gt_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        num_shapes = len(contours)

        # It is not necessary to use torch.tensor since the batch automagically assigns that
        label = num_shapes
        
        if self.transform is not None:
            image = self.transform(image)

        return (image, label)

# image normalization
transform = transforms.Compose([
    transforms.ToTensor()
])

# creació dels conjunts d'entrenament i test
train_ds = AIXI_Shape(img_files, label_files, transform)
train_dl = DataLoader(train_ds, batch_size=64)
train_dl.dataset[0][0].shape

torch.Size([3, 224, 224])

In [4]:
test_ds = AIXI_Shape(test_img_files, test_label_files, transform)
test_dl = DataLoader(test_ds, batch_size=64)
test_dl.dataset[0][0].shape

torch.Size([3, 224, 224])

## Xarxa
Com sempre, vosaltres us encarregau de dissenyar la xarxa:

In [5]:
class MyNet(nn.Module):

    def __init__(self):
        super(MyNet, self).__init__()
        # https://blog.paperspace.com/vgg-from-scratch-pytorch/
        # https://www.kaggle.com/code/datastrophy/vgg16-pytorch-implementation
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1, stride=1), # 224x224x3 -> 224x224x64
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1, stride=1), # 224x224x64 -> 224x224x64
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), # 224x224x64 -> 112x112x64
            
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1, stride=1), # 112x112x64 -> 112x112x128
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1, stride=1), # 112x112x128 -> 112x112x128
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), # 112x112x128 -> 56x56x128
            
            #nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1, stride=1), # 56x56x128 -> 56x56x256
            #nn.ReLU(inplace=True),
            #nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1, stride=1), # 56x56x256 -> 56x56x256
            #nn.ReLU(inplace=True),
            #nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1, stride=1), # 56x56x256 -> 56x56x256
            #nn.ReLU(inplace=True),
            #nn.MaxPool2d(kernel_size=2, stride=2), # 56x56x256 -> 28x28x256
            
            #nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1, stride=1), # 28x28x256 -> 28x28x512
            #nn.ReLU(inplace=True),
            #nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1, stride=1), # 28x28x512 -> 28x28x512
            #nn.ReLU(inplace=True),
            #nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1, stride=1), # 28x28x512 -> 28x28x512
            #nn.ReLU(inplace=True),
            #nn.MaxPool2d(kernel_size=2, stride=2), # 28x28x512 -> 14x14x512
            
            #nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1, stride=1), # 14x14x512 -> 14x14x512
            #nn.ReLU(inplace=True),
            #nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1, stride=1), # 14x14x512 -> 14x14x512
            #nn.ReLU(inplace=True),
            #nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1, stride=1), # 14x14x512 -> 14x14x512
            #nn.ReLU(inplace=True),
            #nn.MaxPool2d(kernel_size=2, stride=2), # 14x14x512 -> 7x7x512           
            
            nn.Conv2d(in_channels=128, out_channels=4, kernel_size=3, padding=1, stride=1),  # 56x56x128 -> 56x56x128
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=56, stride=56)  # 7x7x1 -> 1x1x1
        )

    def forward(self, x):
        x = self.features(x)
        return x

# Entrenament

El blucle d'entrenament és el de sempre. Només heu de pensar quina funció de pèrdua heu d'emprar per el vostre/nostre problema

In [6]:
def train(model, device, train_loader, optimizer, epoch, log_interval=100, verbose=True):
    
    model.train()

    loss_v = 0

    for batch_idx, (data, target) in enumerate(train_loader):
    
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
      
        # Cross entropy needs shapes to match?
        print(output.shape)
        print(target.shape)
        
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
      
        loss_v += loss.item()

    loss_v /= len(train_loader.dataset)
    print('\nTrain set: Average loss: {:.4f}\n'.format(loss_v))
 
    return loss_v


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            
            test_loss += F.cross_entropy(output, target_resized, reduction="sum")
          
   
    test_loss /= len(test_loader.dataset)

    
    return test_loss

## Entrenament

In [7]:
torch.manual_seed(33)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device {device}")

epochs = 15
lr =0.00001

model = MyNet().to(device)

optimizer = optim.Adam(model.parameters(), lr=lr)

# Guardam el valor de pèrdua mig de cada iteració (època)
train_l = np.zeros((epochs))
test_l = np.zeros((epochs))

# Bucle d'entrenament
for epoch in range(0, epochs):
    train_l[epoch] = train(model, device, train_dl, optimizer, epoch)
    test_l[epoch]  = test(model, device, test_dl)


Using device cpu
torch.Size([64, 4, 1, 1])
torch.Size([64])


RuntimeError: only batches of spatial targets supported (3D tensors) but got targets of dimension: 1

## Validació

Heu de fer vosaltres la validació depenent del problema que voldreu resoldre